## PARCEIROS 360 - FATURAMENTO

###  Pagamento - Utilização Factoring & Bloqueios

- <b>Analista(s)</b>: Daniel Rezende
- <b>Projeto</b>: Parceiros 360 - Um modelo de análise de risco para auxiliar no controle das tomadas de decisões através de análises preditivas
- <b>Data</b>: 24/10/2022
- <b>Descrição</b>:

In [14]:
!python --version

Python 3.8.16


### BIBLIOTECAS:

In [15]:
## data
import pandas as pd
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta
from unicodedata import normalize

## vis
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

## modelo
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

from treeinterpreter import treeinterpreter as ti

## alertas
import warnings
warnings.filterwarnings("ignore")

### LAYOUT:

In [16]:
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 6)

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

rc={'font.size': 16, 'axes.labelsize': 16, 'legend.fontsize': 16,
    'axes.titlesize': 18, 'xtick.labelsize': 16, 'ytick.labelsize': 16}

sns.set(rc=rc)
sns.set_style("darkgrid")
sns.set_palette("pastel")

### CONFIGS:

In [17]:
#pd.set_option('max_columns', 200)
pd.options.display.max_columns = 200 

seed = 42
np.random.seed(seed)

### DADOS:

In [18]:
PATH = "datasets/"

#### FATURAMENTO: 

In [19]:
FILE = "02_FATURAMENTO.csv"
faturamento = pd.read_csv(PATH + FILE, sep = ';', encoding = 'ISO-8859-1', engine ='python')
faturamento = faturamento[['RAIZ_CNPJ', 'COMPETENCIA', 
                           'ARRECADACAO_EFETIVA_BOM', 'ARRECADACAO_EFETIVA_MEDIO', 'ARRECADACAO_EFETIVA_RUIM', 
                           'AJUSTE_BOM', 'AJUSTE_MEDIO',
                           'AGING_PGTO_BOM', 'AGING_PGTO_MEDIO', 'AGING_PGTO_RUIM', 'AGING_PGTO_SEM_ANALISE',
                           'PLANTA_BOM', 'PLANTA_MEDIO', 'PLANTA_RUIM', 'PLANTA_SEM_ANALISE',
                           'INADIMPLENCIA_BOM', 'INADIMPLENCIA_MEDIO', 'INADIMPLENCIA_RUIM',
                           'SCORE_RUIM', 'SCORE_MEDIO', 'SCORE_BOM']]

faturamento['COMPETENCIA'] = pd.to_datetime(faturamento['COMPETENCIA'])
faturamento['RAIZ_CNPJ'] = faturamento['RAIZ_CNPJ'].apply(lambda x: str(x).zfill(8))
faturamento = faturamento[faturamento["RAIZ_CNPJ"].str.contains("NAN")== False]
faturamento = faturamento[faturamento["RAIZ_CNPJ"].str.contains("-")== False]

for i in faturamento.columns:
    if faturamento[i].dtypes=='object':
        faturamento[i] = faturamento[i].str.upper()
        
faturamento = faturamento.drop_duplicates().reset_index(drop=True)

faturamento

,RAIZ_CNPJ,COMPETENCIA,ARRECADACAO_EFETIVA_BOM,ARRECADACAO_EFETIVA_MEDIO,ARRECADACAO_EFETIVA_RUIM,AJUSTE_BOM,AJUSTE_MEDIO,AGING_PGTO_BOM,AGING_PGTO_MEDIO,AGING_PGTO_RUIM,AGING_PGTO_SEM_ANALISE,PLANTA_BOM,PLANTA_MEDIO,PLANTA_RUIM,PLANTA_SEM_ANALISE,INADIMPLENCIA_BOM,INADIMPLENCIA_MEDIO,INADIMPLENCIA_RUIM,SCORE_RUIM,SCORE_MEDIO,SCORE_BOM
0,00000000,2021-01-01,1,0,0,1,0,0,1,0,0.0,1,0,0,0.0,1,0,0,0,1,0
1,00000000,2021-02-01,1,0,0,1,0,0,1,0,0.0,0,0,1,0.0,1,0,0,0,1,0
2,00000000,2021-03-01,1,0,0,1,0,0,1,0,0.0,0,0,1,0.0,1,0,0,0,1,0
3,00000000,2021-04-01,1,0,0,1,0,0,1,0,0.0,0,0,1,0.0,1,0,0,0,1,0
4,00000000,2021-05-01,1,0,0,1,0,0,1,0,0.0,0,0,1,0.0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46355,98671597,2022-03-01,1,0,0,0,1,1,0,0,0.0,0,1,0,0.0,1,0,0,0,0,1
46356,98671597,2022-04-01,1,0,0,0,1,1,0,0,0.0,0,1,0,0.0,1,0,0,0,0,1
46357,98671597,2022-05-01,1,0,0,0,1,1,0,0,0.0,1,0,0,0.0,1,0,0,0,0,1
46358,98671597,2022-06-01,1,0,0,0,1,1,0,0,0.0,0,1,0,0.0,1,0,0,0,0,1


***
### BASE DE CADASTRO DE NOME DE FORNECEDORES:

- Essa base contem o cadastro dos fornecedores. Cada fornecedor pode ter mais de um CNPJ (a ideia de filiais) e também cada uma dessas filiais pode ter um código que está armazenado na coluna 'FORNECEDOR'.
- É utilizada para o join com a base de pagamento, pois a base de pagamento não possui a coluna 'RAIZ_CNPJ'.

In [20]:
FILE = "nome_fornecedores.xlsx"
nome_fornecedores = pd.read_excel(PATH + FILE)
nome_fornecedores = nome_fornecedores[['RAIZ_CNPJ', 'NOME_FORNECEDOR', 'FORNECEDOR', 'AREA_ATUACAO']]

nome_fornecedores['RAIZ_CNPJ'] = nome_fornecedores['RAIZ_CNPJ'].apply(lambda x: str(x).zfill(8))
nome_fornecedores = nome_fornecedores[nome_fornecedores["RAIZ_CNPJ"].str.contains("NAN")== False]
nome_fornecedores = nome_fornecedores[nome_fornecedores["RAIZ_CNPJ"].str.contains("-")== False]

for i in nome_fornecedores.columns:
    if nome_fornecedores[i].dtypes=='object':
        nome_fornecedores[i] = nome_fornecedores[i].str.upper()
        
nome_fornecedores = nome_fornecedores.drop_duplicates().reset_index(drop=True)

nome_fornecedores

,RAIZ_CNPJ,NOME_FORNECEDOR,FORNECEDOR,AREA_ATUACAO
0,02714643,333 COMERCIO E COMUNICACOES LTDA,7700312,COMÃRCIO
1,02714643,333 COMERCIO E COMUNICACOES LTDA,7700251,COMÃRCIO
2,02714643,333 COMERCIO E COMUNICACOES LTDA,7716773,COMÃRCIO
3,02714643,333 COMERCIO E COMUNICACOES LTDA,7721785,COMÃRCIO
4,02714643,333 COMERCIO E COMUNICACOES LTDA,7729957,COMÃRCIO
...,...,...,...,...
4572,08747596,ZOTTY CELULARES LTDA - ME,7729259,COMÃRCIO
4573,08747596,ZOTTY CELULARES LTDA - ME,7729332,COMÃRCIO
4574,05216804,ZTE DO BRASIL INDÃSTRIA COMÃRCIO,14017703,TELECOMUNICAÃÃES
4575,05216804,ZTE DO BRASIL INDÃSTRIA COMÃRCIO,14025498,TELECOMUNICAÃÃES


***
### JOIN Faturamento com Nome de Fornecedores

In [21]:
faturamentoj = faturamento.join(nome_fornecedores.set_index('RAIZ_CNPJ'),
                               how='inner',
                               on=['RAIZ_CNPJ'])\
                          .drop_duplicates().reset_index(drop=True)

faturamentoj

,RAIZ_CNPJ,COMPETENCIA,ARRECADACAO_EFETIVA_BOM,ARRECADACAO_EFETIVA_MEDIO,ARRECADACAO_EFETIVA_RUIM,AJUSTE_BOM,AJUSTE_MEDIO,AGING_PGTO_BOM,AGING_PGTO_MEDIO,AGING_PGTO_RUIM,AGING_PGTO_SEM_ANALISE,PLANTA_BOM,PLANTA_MEDIO,PLANTA_RUIM,PLANTA_SEM_ANALISE,INADIMPLENCIA_BOM,INADIMPLENCIA_MEDIO,INADIMPLENCIA_RUIM,SCORE_RUIM,SCORE_MEDIO,SCORE_BOM,NOME_FORNECEDOR,FORNECEDOR,AREA_ATUACAO
0,00000000,2021-01-01,1,0,0,1,0,0,1,0,0.0,1,0,0,0.0,1,0,0,0,1,0,HODNET S/A,9200015,ATIVIDADES FINANCEIRAS
1,00000000,2021-01-01,1,0,0,1,0,0,1,0,0.0,1,0,0,0.0,1,0,0,0,1,0,HODNET S/A,9200031,ATIVIDADES FINANCEIRAS
2,00000000,2021-02-01,1,0,0,1,0,0,1,0,0.0,0,0,1,0.0,1,0,0,0,1,0,HODNET S/A,9200015,ATIVIDADES FINANCEIRAS
3,00000000,2021-02-01,1,0,0,1,0,0,1,0,0.0,0,0,1,0.0,1,0,0,0,1,0,HODNET S/A,9200031,ATIVIDADES FINANCEIRAS
4,00000000,2021-03-01,1,0,0,1,0,0,1,0,0.0,0,0,1,0.0,1,0,0,0,1,0,HODNET S/A,9200015,ATIVIDADES FINANCEIRAS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34005,96534094,2022-07-01,0,1,0,1,0,0,0,1,0.0,1,0,0,0.0,0,0,1,1,0,0,ACCENTURE DO BRASIL LTDA,14025436,CONSULTORIA
34006,96534094,2022-07-01,0,1,0,1,0,0,0,1,0.0,1,0,0,0.0,0,0,1,1,0,0,ACCENTURE DO BRASIL LTDA,14062035,CONSULTORIA
34007,96534094,2022-07-01,0,1,0,1,0,0,0,1,0.0,1,0,0,0.0,0,0,1,1,0,0,ACCENTURE DO BRASIL LTDA,14060244,CONSULTORIA
34008,96534094,2022-07-01,0,1,0,1,0,0,0,1,0.0,1,0,0,0.0,0,0,1,1,0,0,ACCENTURE DO BRASIL LTDA,14024766,CONSULTORIA


In [22]:
# renomeando colunas
faturamentoj2 = faturamentoj.rename(columns={'SCORE_RUIM': 'RUIM'})
faturamentoj2 = faturamentoj2.rename(columns={'SCORE_MEDIO': 'MEDIO'})
faturamentoj2 = faturamentoj2.rename(columns={'SCORE_BOM': 'BOM'})

#juntando colunas em apenas uma coluna categorica
faturamentoj2['ARRECADACAO_EFETIVA'] = faturamentoj2[['ARRECADACAO_EFETIVA_RUIM', 'ARRECADACAO_EFETIVA_MEDIO', 'ARRECADACAO_EFETIVA_BOM']].idxmax(axis=1)
faturamentoj2['AJUSTE'] = faturamentoj2[['AJUSTE_MEDIO', 'AJUSTE_BOM']].idxmax(axis=1)
faturamentoj2['AGING_PGTO'] = faturamentoj2[['AGING_PGTO_RUIM', 'AGING_PGTO_MEDIO', 'AGING_PGTO_BOM', 'AGING_PGTO_SEM_ANALISE']].idxmax(axis=1)
faturamentoj2['PLANTA'] = faturamentoj2[['PLANTA_RUIM', 'PLANTA_MEDIO', 'PLANTA_BOM', 'PLANTA_SEM_ANALISE']].idxmax(axis=1)
faturamentoj2['INADIMPLENCIA'] = faturamentoj2[['INADIMPLENCIA_BOM', 'INADIMPLENCIA_MEDIO', 'INADIMPLENCIA_RUIM']].idxmax(axis=1)
faturamentoj2['KPI_faturamento'] = faturamentoj2[['RUIM', 'MEDIO', 'BOM']].idxmax(axis=1)

#exclui 
faturamentoj3 = faturamentoj2.drop(['RUIM', 'MEDIO', 'BOM',\
                                    'ARRECADACAO_EFETIVA_RUIM', 'ARRECADACAO_EFETIVA_MEDIO', 'ARRECADACAO_EFETIVA_BOM',\
                                    'AJUSTE_MEDIO', 'AJUSTE_BOM',\
                                    'AGING_PGTO_RUIM', 'AGING_PGTO_MEDIO', 'AGING_PGTO_BOM', 'AGING_PGTO_SEM_ANALISE',\
                                    'PLANTA_RUIM', 'PLANTA_MEDIO', 'PLANTA_BOM', 'PLANTA_SEM_ANALISE',\
                                    'INADIMPLENCIA_BOM', 'INADIMPLENCIA_MEDIO', 'INADIMPLENCIA_RUIM'], axis=1)

In [23]:
faturamentoj3

,RAIZ_CNPJ,COMPETENCIA,NOME_FORNECEDOR,FORNECEDOR,AREA_ATUACAO,ARRECADACAO_EFETIVA,AJUSTE,AGING_PGTO,PLANTA,INADIMPLENCIA,KPI_faturamento
0,00000000,2021-01-01,HODNET S/A,9200015,ATIVIDADES FINANCEIRAS,ARRECADACAO_EFETIVA_BOM,AJUSTE_BOM,AGING_PGTO_MEDIO,PLANTA_BOM,INADIMPLENCIA_BOM,MEDIO
1,00000000,2021-01-01,HODNET S/A,9200031,ATIVIDADES FINANCEIRAS,ARRECADACAO_EFETIVA_BOM,AJUSTE_BOM,AGING_PGTO_MEDIO,PLANTA_BOM,INADIMPLENCIA_BOM,MEDIO
2,00000000,2021-02-01,HODNET S/A,9200015,ATIVIDADES FINANCEIRAS,ARRECADACAO_EFETIVA_BOM,AJUSTE_BOM,AGING_PGTO_MEDIO,PLANTA_RUIM,INADIMPLENCIA_BOM,MEDIO
3,00000000,2021-02-01,HODNET S/A,9200031,ATIVIDADES FINANCEIRAS,ARRECADACAO_EFETIVA_BOM,AJUSTE_BOM,AGING_PGTO_MEDIO,PLANTA_RUIM,INADIMPLENCIA_BOM,MEDIO
4,00000000,2021-03-01,HODNET S/A,9200015,ATIVIDADES FINANCEIRAS,ARRECADACAO_EFETIVA_BOM,AJUSTE_BOM,AGING_PGTO_MEDIO,PLANTA_RUIM,INADIMPLENCIA_BOM,MEDIO
...,...,...,...,...,...,...,...,...,...,...,...
34005,96534094,2022-07-01,ACCENTURE DO BRASIL LTDA,14025436,CONSULTORIA,ARRECADACAO_EFETIVA_MEDIO,AJUSTE_BOM,AGING_PGTO_RUIM,PLANTA_BOM,INADIMPLENCIA_RUIM,RUIM
34006,96534094,2022-07-01,ACCENTURE DO BRASIL LTDA,14062035,CONSULTORIA,ARRECADACAO_EFETIVA_MEDIO,AJUSTE_BOM,AGING_PGTO_RUIM,PLANTA_BOM,INADIMPLENCIA_RUIM,RUIM
34007,96534094,2022-07-01,ACCENTURE DO BRASIL LTDA,14060244,CONSULTORIA,ARRECADACAO_EFETIVA_MEDIO,AJUSTE_BOM,AGING_PGTO_RUIM,PLANTA_BOM,INADIMPLENCIA_RUIM,RUIM
34008,96534094,2022-07-01,ACCENTURE DO BRASIL LTDA,14024766,CONSULTORIA,ARRECADACAO_EFETIVA_MEDIO,AJUSTE_BOM,AGING_PGTO_RUIM,PLANTA_BOM,INADIMPLENCIA_RUIM,RUIM


***
### ORDINAL ENCODING

In [24]:
from sklearn.preprocessing import OrdinalEncoder

faturamentoj3_oe = OrdinalEncoder().fit_transform(faturamentoj3[['ARRECADACAO_EFETIVA', 'AJUSTE', 'AGING_PGTO', 'PLANTA', 'INADIMPLENCIA']])

faturamentoj3['ARRECADACAO_EFETIVA_oe'] = faturamentoj3_oe[:, 0]

faturamentoj3['AJUSTE_oe'] = faturamentoj3_oe[:, 1]

faturamentoj3['AGING_PGTO_oe'] = faturamentoj3_oe[:, 2]

faturamentoj3['PLANTA_oe'] = faturamentoj3_oe[:, 3]

faturamentoj3['INADIMPLENCIA_oe'] = faturamentoj3_oe[:, 4]

In [25]:
faturamentoj3.columns

Index(['RAIZ_CNPJ', 'COMPETENCIA', 'NOME_FORNECEDOR', 'FORNECEDOR', 'AREA_ATUACAO', 'ARRECADACAO_EFETIVA', 'AJUSTE', 'AGING_PGTO', 'PLANTA', 'INADIMPLENCIA', 'KPI_faturamento', 'ARRECADACAO_EFETIVA_oe', 'AJUSTE_oe', 'AGING_PGTO_oe', 'PLANTA_oe', 'INADIMPLENCIA_oe'], dtype='object')

In [26]:
faturamentoj3.to_excel(r'intermediate/etl_previsao_faturamento.xlsx')